In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
df = pd.read_csv(r"C:\Users\A\Documents\Latihan Cleaning\Copy of Data Science pertemuan 1 (Responses) - Form Responses 1.csv")

In [ ]:
def clean_residence(value):
    value = str(value).lower()  # Ubah ke huruf kecil agar seragam
    
    if "rumah" in value:
        return "Rumah"
    elif "kost" in value or "kos" in value:
        return "Kost"
    elif "kantor" in value or "mess" in value:
        return "Mess/Kantor"
    else:
        return "Lainnya"

df['Tinggal dirumah / kost ?'] = df['Tinggal dirumah / kost ?'].apply(clean_residence)

In [ ]:
print(df['Tinggal dirumah / kost ?'].value_counts())

In [ ]:
def clean_transport(transport):
    transport = transport.lower().strip()
    
    if any(x in transport for x in ['motor', 'sepeda', 'kendaraan roda 2', 'kendaraan bermotor']):
        return 'Motor'
    elif any(x in transport for x in ['jalan kaki', 'sikel']):
        return 'Jalan Kaki'
    elif any(x in transport for x in ['mobil', 'kendaraan pribadi']):
        return 'Mobil'
    elif any(x in transport for x in ['grab', 'gojek', 'maxim', 'transmusi']):
        return 'Transportasi Online'
    else:
        return 'Lainnya'

df['Jenis Transportasi'] = df['Apa jenis transportasi yang biasa Anda gunakan untuk pergi ke Kampus?'].apply(clean_transport)

In [ ]:
print(df['Jenis Transportasi'].value_counts())

In [ ]:
def clean_sleep_hours(sleep):
    sleep = str(sleep).lower().strip()
    
    # Jika mengandung kata tidak tentu atau sejenisnya, anggap NaN
    if any(x in sleep for x in ['tidak tentu', 'belum tentu']):
        return np.nan

    # Ambil semua angka dari string
    numbers = re.findall(r'\d+', sleep)
    numbers = list(map(int, numbers))

    if len(numbers) == 1:
        return numbers[0]  # Jika hanya satu angka, langsung gunakan
    elif len(numbers) > 1:
        return sum(numbers) / len(numbers)  # Jika rentang, ambil rata-rata
    else:
        return np.nan  # Jika tidak ada angka, anggap NaN

df['Jam Tidur'] = df['Berapa jam tidur yang Anda dapatkan setiap malam pada hari kuliah?'].apply(clean_sleep_hours)


In [ ]:
print(df['Jam Tidur'].value_counts())

In [ ]:
def categorize_study_hours(value):
    try:
        num_hours = float(str(value).replace(" jam", "").replace(",", "."))  # Convert ke float
        if num_hours < 1:
            return "<1 jam"
        elif 1 <= num_hours <= 2:
            return "1 - 2 jam"
        elif 3 <= num_hours <= 4:
            return "3 - 4 jam"
        else:
            return ">4 jam"
    except:
        return "Lainnya"  # Untuk yang tidak bisa dikonversi ke angka

df["Berapa jam belajar di luar jam kuliah setiap harinya?"] = df["Berapa jam belajar di luar jam kuliah setiap harinya?"].apply(categorize_study_hours)

In [ ]:
print(df["Berapa jam belajar di luar jam kuliah setiap harinya?"].value_counts())

In [ ]:
bins = [0, 17, 24, 31, 40, 100] 
labels = ["<17", "17-24", "25-31", "32-40", "40+"]  

In [ ]:
df["Kategori Umur"] = pd.cut(df['Umur Anda (tulis angka saja)'], bins=bins, labels=labels, right=True)
print(df[['Umur Anda (tulis angka saja)', "Kategori Umur"]])
print(df['Umur Anda (tulis angka saja)'])

In [ ]:
def convert_to_minutes(time_str):
    time_str = str(time_str).lower()  # Ubah ke huruf kecil
    numbers = re.findall(r'\d+', time_str)  # Cari semua angka

    if not numbers:  
        return None  # Jika tidak ada angka, return None

    numbers = list(map(int, numbers))  # Konversi ke integer
    
    if "jam" in time_str:
        jam = numbers[0]
        menit = numbers[1] if len(numbers) > 1 else 0  # Jika tidak ada menit, anggap 0
        return jam * 60 + menit  # Konversi ke menit

    return numbers[0]  # Jika hanya angka tanpa "jam", anggap dalam menit

# Pastikan df sudah ada sebelum kode ini dijalankan
df['Waktu Tempuh (Menit)'] = df['Berapa lama waktu tempuh dari rumah ke kampus?'].apply(convert_to_minutes)

In [ ]:
print(df[['Berapa lama waktu tempuh dari rumah ke kampus?', 'Waktu Tempuh (Menit)']])

In [ ]:
# waktu belajar

#ubah format waktu belajar ke menit
def convert_learning_time(time_str):
    time_str = str(time_str).lower()  # Ubah ke string dan lowercase
    numbers = re.findall(r'\d+', time_str)  # Ambil angka dari string

    if not numbers:  # Jika tidak ada angka, kembalikan NaN
        return None  

    if 'jam' in time_str:  # Jika ada kata 'jam'
        jam = int(numbers[0])  # Ambil angka pertama sebagai jam
        menit = int(numbers[1]) if len(numbers) > 1 else 0  # Jika ada angka kedua, anggap sebagai menit
        return jam * 60 + menit  # Konversi ke menit

    elif 'menit' in time_str or len(numbers) == 1:  
        return int(numbers[0])  # Anggap langsung dalam menit

    return None  # Jika tidak cocok dengan format di atas

# Terapkan fungsi pada kolom target
df['Waktu Belajar (Menit)'] = df['Berapa jam belajar di luar jam kuliah setiap harinya?'].apply(convert_learning_time)



In [ ]:
print(df[['Berapa jam belajar di luar jam kuliah setiap harinya?', 'Waktu Belajar (Menit)']])

In [ ]:
df = df.dropna(subset=['Waktu Belajar (Menit)'])
df['Waktu Belajar (Menit)'] = df['Waktu Belajar (Menit)'].astype(int)
print(df['Waktu Belajar (Menit)'])

In [ ]:
def clean_club(value):
    # Standarisasi teks (huruf kecil semua)
    value = str(value).lower()

    # Mapping typo ke nama yang benar
    replacements = {
        "digiepreneur": "digipreneur",
        "digipreanur": "digipreneur",
        "digiprener": "digipreneur",
        "digipreneur club": "digipreneur",
        "codingan": "coding",
        "club fotografi": "fotografi",
        "photografi": "fotografi",
        "photography": "fotografi",
        "club esport": "esport",
        "e sport": "esport",
        "club coding": "coding",
        "iya, coding": "coding",
        "ya, coding": "coding",
        "iya": "lainnya",
        "ya": "lainnya",
        "coding club": "coding",
        "ya, IOT dan coding": "iot, coding",
        "pbc": "lainnya",
        "broadcasting club": "broadcasting",
        "club broadcasting": "broadcasting",
        "cinecomtech": "cinecomtech"
    }

    # Standarisasi typo
    for typo, correct in replacements.items():
        if typo in value:
            return correct

    # Jika ada lebih dari satu club yang valid, pisahkan
    valid_clubs = {"esport", "coding", "digipreneur", "cinecomtech", "broadcasting", "iot", "fotografi"}
    result = [club for club in valid_clubs if club in value]

    if result:
        return ", ".join(result)  # Gabungkan jika ada lebih dari satu club
    elif "tidak" in value or "-" in value:
        return "tidak"
    else:
        return "lainnya"

# Terapkan ke dataframe
df["club_bersih"] = df["Apakah Anda mengikuti kegiatan club? Jika ya, sebutkan kegiatannya"].apply(clean_club)

In [ ]:
print(df["club_bersih"].value_counts())

In [ ]:
def clean_stress(value):
    value = str(value).lower().strip()

    # Mapping kategori stres
    sangat_sering = ["sangat sering", "setiap hari", "selalu", "setiap saat", "hampir setiap hari"]
    sering = ["sering", "sering banget", "cukup sering", "lumayan sering", "sering sekali", "sering stres"]
    kadang_kadang = ["kadang", "kadang-kadang", "kadang kadang", "terkadang"]
    jarang = ["jarang", "cukup jarang"]
    tidak_pernah = ["tidak pernah", "tidak", "tidak terlalu", "tidak terlalu sering", "tidak sering"]

    # Membersihkan input ke dalam kategori yang sesuai
    if value in sangat_sering or "setiap" in value:
        return "Sangat Sering"
    elif value in sering or "banyak tugas" in value:
        return "Sering"
    elif value in kadang_kadang or "kadang" in value:
        return "Kadang-kadang"
    elif value in jarang:
        return "Jarang"
    elif value in tidak_pernah:
        return "Tidak Pernah"
    else:
        return "Lainnya"  # Untuk jawaban seperti "mumet", "enjoy", atau yang tidak jelas

df["stres_bersih"] = df["Seberapa sering Anda merasa stres karena kuliah?"].apply(clean_stress)

In [ ]:
print(df["stres_bersih"].value_counts())

In [ ]:
def clean_friends(value):
    value = str(value).lower().strip()

    # Mengubah jawaban verbal menjadi angka
    if "banyak" in value or "semua" in value or "ada banyak" in value:
        return "Lebih dari 10"
    elif "tidak ada" in value or "ga ada" in value or "tidak punya" in value:
        return "0"
    elif "1" in value and not ("10" in value or "15" in value):
        return "1-5"
    elif "2" in value and not ("20" in value):
        return "1-5"
    elif "3" in value:
        return "1-5"
    elif "4" in value:
        return "1-5"
    elif "5" in value:
        return "1-5"
    elif "6" in value:
        return "6-10"
    elif "7" in value:
        return "6-10"
    elif "8" in value:
        return "6-10"
    elif "9" in value:
        return "6-10"
    elif "10" in value or "+/- 10" in value or "lebih krg hampir 10" in value:
        return "6-10"
    elif "11" in value or "lebih dari 10" in value:
        return "Lebih dari 10"
    elif "15" in value or "20" in value or "30" in value or "1000" in value:
        return "Lebih dari 10"
    else:
        return "Lainnya"  # Untuk jawaban yang tidak jelas

# Terapkan ke dataframe
df["jumlah_teman_bersih"] = df["Berapa jumlah teman dekat yang Anda miliki di kampus?"].apply(clean_friends)


In [ ]:
print(df["jumlah_teman_bersih"].value_counts())

In [ ]:
def clean_engagement(value):
    value = str(value).lower().strip()

    # Mapping jawaban ke kategori utama
    if "rendah" in value:
        return "Rendah"
    elif "tinggi" in value or "cukup tinggi" in value or "di atas sedang" in value:
        return "Tinggi"
    elif "sedang" in value or "lumayan" in value:
        return "Sedang"
    else:
        return "Lainnya"  # Untuk jawaban yang tidak jelas atau ambigu

df["keterlibatan_bersih"] = df["Bagaimana Anda menilai tingkat keterlibatan Anda dalam pembelajaran di kelas ini (rendah, sedang, tinggi)?"].apply(clean_engagement)

In [ ]:
print(df["keterlibatan_bersih"].value_counts())

In [ ]:
# Hapus kolom lama
df.drop(columns=[
    "Bagaimana Anda menilai tingkat keterlibatan Anda dalam pembelajaran di kelas ini (rendah, sedang, tinggi)?",
    "Berapa jumlah teman dekat yang Anda miliki di kampus?",
    "Seberapa sering Anda merasa stres karena kuliah?",
    "Apakah Anda mengikuti kegiatan club? Jika ya, sebutkan kegiatannya",
    "Apa jenis transportasi yang biasa Anda gunakan untuk pergi ke Kampus?",
    "Berapa jam belajar di luar jam kuliah setiap harinya?",
    "Umur Anda (tulis angka saja)",
    "Berapa lama waktu tempuh dari rumah ke kampus?",
    "Berapa jam tidur yang Anda dapatkan setiap malam pada hari kuliah?",
    "Timestamp",
], inplace=True)

# Ganti nama kolom bersih ke nama asli
df.rename(columns={
    "keterlibatan_bersih": "Bagaimana Anda menilai tingkat keterlibatan Anda dalam pembelajaran di kelas ini (rendah, sedang, tinggi)?",
    "stres_bersih": "Seberapa sering Anda merasa stres karena kuliah?",
    "club_bersih": "Apakah Anda mengikuti kegiatan club? Jika ya, sebutkan kegiatannya",
    "Jenis Transportasi": "Apa jenis transportasi yang biasa Anda gunakan untuk pergi ke Kampus?",
    "Waktu Belajar (Menit)": "Berapa jam belajar di luar jam kuliah setiap harinya?",
    "Kategori Umur": "Umur Anda (tulis angka saja)",
    "Waktu Tempuh (Menit)": "Berapa lama waktu tempuh dari rumah ke kampus?",
    "Jam Tidur": "Berapa jam tidur yang Anda dapatkan setiap malam pada hari kuliah?",
    "jumlah_teman_bersih": "Berapa jumlah teman dekat yang Anda miliki di kampus?", 
}, inplace=True)


In [ ]:
#cek data

In [ ]:
print(df.isna().sum())  # Lihat berapa banyak nilai NaN di tiap kolom

In [ ]:
print(df.dtypes)


In [ ]:
print(df)

In [ ]:
df.to_csv('hasil_klasifikasi.csv', index=False)
print("Data telah di-clean dan disimpan")